In [1]:
import os
import cv2
import pathlib
import numpy as np
import re
import imageio
from pathlib import Path

In [2]:
# Set the folder path where images are located and the tile size
path = pathlib.Path(os.getcwd())
Hagen_testmix = path/'Hagen_testmix'
tile_size = 256
samples = ['actin-20x-noise1','actin-60x-noise1','actin-60x-noise2','mito-20x-noise1','mito-60x-noise1','mito-60x-noise2',
           'actin-confocal','mito-confocal','nucleus','membrane']

In [3]:
def recreate_image_from_patches(patches, img_size, overlap, patch_size=256):
    
    reconstructed_image = np.zeros((img_size, img_size), dtype=np.float32) 
    patch_count = np.zeros((img_size, img_size), dtype=np.float32)

    step = patch_size - overlap
    idx = 0
    
    for y in range(0, img_size - patch_size + 1, step):
        for x in range(0, img_size - patch_size + 1, step):
            reconstructed_image[y:y + patch_size, x:x + patch_size] += patches[idx]
            patch_count[y:y + patch_size, x:x + patch_size] += 1
            idx += 1
    
    # Avoid division by zero
    patch_count[patch_count == 0] = 1e-10
    
    reconstructed_image //= patch_count  # Averaging the pixel values    
    return reconstructed_image

In [4]:
def natural_sort(l): 
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)', key)]
    return sorted(l, key=alphanum_key)

In [5]:
def reconstruct(folder_path, original_filename, original_shape, folder_out, tag):
    # Reconstructs an image from its tiles   
    new_folder = Path(folder_path)
    new_folder = os.path.join(new_folder.parent.absolute(),f'{folder_out}_images',tag)
    os.makedirs(new_folder,exist_ok=True)
    tiles = []
    patch_size = 256

    img_size = original_shape[0]
    if img_size==2048:
        overlap = 128        # 32 as option
    elif img_size==1024:
        overlap = 128        # 64 as option
    else:
        overlap = 128
    
    # Setting the number of patches based on size
    number_of_patches = ((img_size - overlap) // (patch_size - overlap))**2
    
    # Sort based on tile number to propperly assign patch locations
    files = natural_sort(os.listdir(folder_path))
    
    for filename in files:
        if os.path.splitext(original_filename)[0] in filename:            
            tile_path = os.path.join(folder_path, filename)
            tile = imageio.v2.imread(tile_path)            
            tiles.append(tile)    
    
    reconstructed_image = recreate_image_from_patches(tiles, img_size, overlap)
    # Save the reconstructed image
    output_image_path = os.path.join(new_folder, f'{original_filename}')
    cv2.imwrite(output_image_path, reconstructed_image.astype(np.uint16))

In [6]:
# Process images of various samples for Basemodel 
for i in samples:
    print(f'working on {i} images')
    fn = sorted(os.listdir(Hagen_testmix/f'gt_images/{i}'))
    for j in fn:
        ref = imageio.v2.imread(Hagen_testmix/f'gt_images/{i}/{j}')        
        reconstruct(Hagen_testmix/'Hagen_pred_tiles', j, np.shape(ref), 'Hagen_pred',i)

working on actin-20x-noise1 images


/home/david/mambaforge/lib/python3.10/site-packages/imageio/plugins/pillow.py:320: UserWarning: Loading 16-bit (uint16) PNG as int32 due to limitations in pillow's PNG decoder. This will be fixed in a future version of pillow which will make this warning dissapear.
  warnings.warn(


working on actin-60x-noise1 images
working on actin-60x-noise2 images
working on mito-20x-noise1 images
working on mito-60x-noise1 images
working on mito-60x-noise2 images
working on actin-confocal images
working on mito-confocal images
working on nucleus images
working on membrane images


In [7]:
# Process images of various samples for Transfer model 
for i in samples:
    print(f'working on {i} images')
    fn = sorted(os.listdir(Hagen_testmix/f'gt_images/{i}'))
    for j in fn:
        ref = imageio.v2.imread(Hagen_testmix/f'gt_images/{i}/{j}')        
        reconstruct(Hagen_testmix/'Transfer_pred_tiles', j, np.shape(ref), 'Transfer_pred',i)

working on actin-20x-noise1 images
working on actin-60x-noise1 images
working on actin-60x-noise2 images
working on mito-20x-noise1 images
working on mito-60x-noise1 images
working on mito-60x-noise2 images
working on actin-confocal images
working on mito-confocal images
working on nucleus images
working on membrane images


In [8]:
# Process images of various samples for a Transfer model using all Hagen images
for i in samples:
    print(f'working on {i} images')
    fn = sorted(os.listdir(Hagen_testmix/f'gt_images/{i}'))
    for j in fn:
        ref = imageio.v2.imread(Hagen_testmix/f'gt_images/{i}/{j}')        
        reconstruct(Hagen_testmix/'Transfer_all_pred_tiles', j, np.shape(ref), 'Transfer_all_pred',i)

working on actin-20x-noise1 images
working on actin-60x-noise1 images
working on actin-60x-noise2 images
working on mito-20x-noise1 images
working on mito-60x-noise1 images
working on mito-60x-noise2 images
working on actin-confocal images
working on mito-confocal images
working on nucleus images
working on membrane images
